# ESN Progression Modeling Lab

## Echo State Networks for Temporal Disease Progression Prediction

This lab implements Echo State Networks (ESN) for predicting the temporal progression of skin conditions. ESNs are a type of recurrent neural network particularly suited for time series prediction due to their reservoir computing approach.

### Key Capabilities

- **Disease Progression Prediction**: Forecast condition severity over time
- **Treatment Response Modeling**: Predict how patients respond to treatments
- **Pattern Recognition**: Identify flare cycles and seasonal patterns
- **Healing Time Estimation**: Estimate recovery trajectories

### Why Echo State Networks?

ESNs are ideal for medical time series because:
1. **Fast Training**: Only output weights need training
2. **Temporal Memory**: Reservoir captures temporal dynamics
3. **Stability**: Well-understood dynamics with spectral radius control
4. **Interpretability**: Easier to analyze than deep networks

In [ ]:
# Environment Setup
import os
import json
import numpy as np
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass, field
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load environment
from dotenv import load_dotenv
load_dotenv()

GATEWAY_URL = os.getenv('REGIMA_GATEWAY_URL', 'http://localhost:3000')
API_KEY = os.getenv('REGIMA_API_KEY', 'demo-key')

# Set random seed for reproducibility
np.random.seed(42)

## 1. Data Structures

Define the data structures for progression modeling.

In [ ]:
@dataclass
class ProgressionDataPoint:
    """A single observation in the progression timeline"""
    timestamp: datetime
    severity: float  # 0.0 (clear) to 1.0 (severe)
    treatment_active: bool = False
    treatment_type: Optional[str] = None
    environmental_factors: Dict[str, float] = field(default_factory=dict)
    
    def to_vector(self) -> np.ndarray:
        """Convert to feature vector for ESN input"""
        # Day of year (normalized) - captures seasonality
        day_of_year = self.timestamp.timetuple().tm_yday / 365.0
        
        # Day of week (normalized) - captures weekly patterns
        day_of_week = self.timestamp.weekday() / 6.0
        
        # Environmental factors
        humidity = self.environmental_factors.get('humidity', 0.5)
        temperature = self.environmental_factors.get('temperature', 0.5)
        stress_level = self.environmental_factors.get('stress', 0.3)
        
        return np.array([
            self.severity,
            float(self.treatment_active),
            day_of_year,
            day_of_week,
            humidity,
            temperature,
            stress_level
        ])

@dataclass
class ProgressionPrediction:
    """A prediction for future progression"""
    timestamp: datetime
    predicted_severity: float
    confidence: float
    lower_bound: float
    upper_bound: float
    
    def to_dict(self) -> Dict:
        return {
            "date": self.timestamp.isoformat(),
            "predicted_severity": round(self.predicted_severity, 4),
            "confidence": round(self.confidence, 4),
            "confidence_interval": {
                "lower": round(self.lower_bound, 4),
                "upper": round(self.upper_bound, 4)
            }
        }

@dataclass
class PatientProgression:
    """Complete progression history for a patient"""
    patient_id: str
    condition: str
    data_points: List[ProgressionDataPoint]
    
    def to_matrix(self) -> np.ndarray:
        """Convert to input matrix for ESN"""
        return np.array([dp.to_vector() for dp in self.data_points])
    
    def get_severities(self) -> np.ndarray:
        """Get severity values as target array"""
        return np.array([dp.severity for dp in self.data_points])

print("Data structures loaded successfully")

## 2. Echo State Network Implementation

Core ESN implementation for temporal progression modeling.

In [ ]:
class EchoStateNetwork:
    """Echo State Network for skin condition progression prediction"""
    
    def __init__(
        self,
        input_size: int = 7,
        reservoir_size: int = 500,
        output_size: int = 1,
        spectral_radius: float = 0.95,
        input_scaling: float = 0.5,
        leaking_rate: float = 0.3,
        regularization: float = 1e-6,
        sparsity: float = 0.1
    ):
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.spectral_radius = spectral_radius
        self.input_scaling = input_scaling
        self.leaking_rate = leaking_rate
        self.regularization = regularization
        self.sparsity = sparsity
        
        # Initialize reservoir
        self._initialize_weights()
        
        # Output weights (to be trained)
        self.W_out = None
        
        # Training state
        self.is_trained = False
        self.training_error = None
    
    def _initialize_weights(self):
        """Initialize reservoir and input weights"""
        # Sparse reservoir weight matrix
        W_reservoir = np.random.randn(self.reservoir_size, self.reservoir_size)
        mask = np.random.rand(self.reservoir_size, self.reservoir_size) < self.sparsity
        W_reservoir *= mask
        
        # Scale to desired spectral radius
        eigenvalues = np.linalg.eigvals(W_reservoir)
        current_spectral_radius = np.max(np.abs(eigenvalues))
        if current_spectral_radius > 0:
            W_reservoir *= self.spectral_radius / current_spectral_radius
        
        self.W_reservoir = W_reservoir
        
        # Input weight matrix
        self.W_in = np.random.randn(self.reservoir_size, self.input_size) * self.input_scaling
        
        # Bias
        self.bias = np.random.randn(self.reservoir_size) * 0.1
    
    def _update_reservoir(self, state: np.ndarray, input_vector: np.ndarray) -> np.ndarray:
        """Update reservoir state with leaky integration"""
        pre_activation = (
            np.dot(self.W_reservoir, state) +
            np.dot(self.W_in, input_vector) +
            self.bias
        )
        new_state = np.tanh(pre_activation)
        
        # Leaky integration
        return (1 - self.leaking_rate) * state + self.leaking_rate * new_state
    
    def _collect_states(self, inputs: np.ndarray, washout: int = 50) -> np.ndarray:
        """Collect reservoir states for all inputs"""
        T = inputs.shape[0]
        states = np.zeros((T, self.reservoir_size))
        state = np.zeros(self.reservoir_size)
        
        for t in range(T):
            state = self._update_reservoir(state, inputs[t])
            states[t] = state
        
        # Return states after washout period
        return states[washout:]
    
    def fit(self, inputs: np.ndarray, targets: np.ndarray, washout: int = 50):
        """Train the ESN on time series data"""
        # Collect reservoir states
        states = self._collect_states(inputs, washout)
        targets_trimmed = targets[washout:]
        
        # Add bias to states
        extended_states = np.hstack([states, np.ones((states.shape[0], 1))])
        
        # Ridge regression for output weights
        reg_matrix = self.regularization * np.eye(extended_states.shape[1])
        self.W_out = np.linalg.solve(
            extended_states.T @ extended_states + reg_matrix,
            extended_states.T @ targets_trimmed
        )
        
        # Calculate training error
        predictions = extended_states @ self.W_out
        self.training_error = np.mean((predictions - targets_trimmed) ** 2)
        self.is_trained = True
        
        return self
    
    def predict(
        self,
        initial_inputs: np.ndarray,
        horizon: int,
        return_confidence: bool = True
    ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """Predict future progression"""
        if not self.is_trained:
            raise ValueError("ESN must be trained before prediction")
        
        # Warm up reservoir with initial inputs
        state = np.zeros(self.reservoir_size)
        for t in range(len(initial_inputs)):
            state = self._update_reservoir(state, initial_inputs[t])
        
        # Generate predictions
        predictions = []
        last_input = initial_inputs[-1].copy()
        
        for _ in range(horizon):
            state = self._update_reservoir(state, last_input)
            extended_state = np.append(state, 1)  # Add bias
            pred = extended_state @ self.W_out
            pred = np.clip(pred, 0, 1)  # Severity bounded [0, 1]
            predictions.append(pred)
            
            # Update input with prediction for next step
            last_input[0] = pred  # Severity is first feature
        
        predictions = np.array(predictions)
        
        if return_confidence:
            # Estimate confidence based on prediction horizon
            confidence = self._estimate_confidence(horizon)
            return predictions, confidence
        
        return predictions, None
    
    def _estimate_confidence(self, horizon: int) -> np.ndarray:
        """Estimate prediction confidence that decays with horizon"""
        base_confidence = 0.95 - np.sqrt(self.training_error) * 0.5
        decay_rate = 0.02  # 2% confidence decay per step
        
        confidence = np.array([
            max(0.3, base_confidence * np.exp(-decay_rate * t))
            for t in range(horizon)
        ])
        
        return confidence

print("ESN implementation loaded")
print(f"Reservoir size: 500, Spectral radius: 0.95")

## 3. Progression Predictor

High-level interface for dermatological progression prediction.

In [ ]:
class ProgressionPredictor:
    """Dermatological progression predictor using ESN"""
    
    def __init__(self, esn_config: Dict = None):
        config = esn_config or {}
        self.esn = EchoStateNetwork(
            reservoir_size=config.get('reservoir_size', 500),
            spectral_radius=config.get('spectral_radius', 0.95),
            input_scaling=config.get('input_scaling', 0.5),
            leaking_rate=config.get('leaking_rate', 0.3),
            regularization=config.get('regularization', 1e-6)
        )
        self.condition_baselines = self._load_condition_baselines()
    
    def _load_condition_baselines(self) -> Dict:
        """Load baseline progression patterns for conditions"""
        return {
            'acne_vulgaris': {
                'typical_duration_weeks': 12,
                'natural_improvement_rate': 0.05,
                'treatment_improvement_rate': 0.15,
                'flare_probability': 0.2,
                'seasonal_factors': {'summer': 0.1, 'winter': -0.05}
            },
            'psoriasis': {
                'typical_duration_weeks': 24,
                'natural_improvement_rate': 0.02,
                'treatment_improvement_rate': 0.10,
                'flare_probability': 0.3,
                'seasonal_factors': {'winter': 0.15, 'summer': -0.10}
            },
            'eczema': {
                'typical_duration_weeks': 8,
                'natural_improvement_rate': 0.03,
                'treatment_improvement_rate': 0.12,
                'flare_probability': 0.25,
                'seasonal_factors': {'winter': 0.20, 'summer': -0.05}
            },
            'rosacea': {
                'typical_duration_weeks': 16,
                'natural_improvement_rate': 0.01,
                'treatment_improvement_rate': 0.08,
                'flare_probability': 0.35,
                'seasonal_factors': {'summer': 0.15, 'winter': 0.05}
            }
        }
    
    def train(self, progression: PatientProgression, washout: int = 10):
        """Train ESN on patient progression data"""
        inputs = progression.to_matrix()
        targets = progression.get_severities().reshape(-1, 1)
        
        # Adjust washout based on data length
        washout = min(washout, len(inputs) // 3)
        
        self.esn.fit(inputs, targets, washout=washout)
        print(f"Training complete. MSE: {self.esn.training_error:.6f}")
    
    def predict_progression(
        self,
        progression: PatientProgression,
        horizon_days: int = 30,
        treatment_plan: Dict = None
    ) -> List[ProgressionPrediction]:
        """Predict future progression"""
        inputs = progression.to_matrix()
        
        # Predict
        predictions, confidence = self.esn.predict(inputs, horizon_days)
        
        # Apply condition-specific adjustments
        baseline = self.condition_baselines.get(progression.condition, {})
        
        # Create prediction objects
        results = []
        last_date = progression.data_points[-1].timestamp
        
        for day in range(horizon_days):
            pred_date = last_date + timedelta(days=day + 1)
            pred_value = float(predictions[day])
            conf = float(confidence[day])
            
            # Apply treatment effect if specified
            if treatment_plan and treatment_plan.get('active'):
                improvement_rate = baseline.get('treatment_improvement_rate', 0.1)
                pred_value = max(0, pred_value - improvement_rate * (day / 30))
            
            # Calculate confidence interval
            interval_width = (1 - conf) * 0.5
            lower = max(0, pred_value - interval_width)
            upper = min(1, pred_value + interval_width)
            
            results.append(ProgressionPrediction(
                timestamp=pred_date,
                predicted_severity=pred_value,
                confidence=conf,
                lower_bound=lower,
                upper_bound=upper
            ))
        
        return results
    
    def estimate_healing_time(
        self,
        progression: PatientProgression,
        target_severity: float = 0.1,
        max_days: int = 180
    ) -> Dict:
        """Estimate time to reach target severity"""
        predictions = self.predict_progression(progression, horizon_days=max_days)
        
        healing_day = None
        for i, pred in enumerate(predictions):
            if pred.predicted_severity <= target_severity:
                healing_day = i + 1
                break
        
        if healing_day:
            return {
                'estimated_days': healing_day,
                'target_severity': target_severity,
                'confidence': predictions[healing_day - 1].confidence,
                'target_date': predictions[healing_day - 1].timestamp.isoformat(),
                'status': 'healing_expected'
            }
        else:
            return {
                'estimated_days': None,
                'target_severity': target_severity,
                'status': 'healing_not_predicted_in_window',
                'final_predicted_severity': predictions[-1].predicted_severity,
                'recommendation': 'Consider treatment adjustment'
            }
    
    def analyze_patterns(
        self,
        progression: PatientProgression
    ) -> Dict:
        """Analyze temporal patterns in progression"""
        severities = progression.get_severities()
        
        # Basic statistics
        stats = {
            'mean_severity': float(np.mean(severities)),
            'std_severity': float(np.std(severities)),
            'max_severity': float(np.max(severities)),
            'min_severity': float(np.min(severities)),
            'trend': self._calculate_trend(severities)
        }
        
        # Detect flares
        flares = self._detect_flares(severities)
        
        # Detect periodicity
        periodicity = self._detect_periodicity(severities)
        
        return {
            'statistics': stats,
            'flares': flares,
            'periodicity': periodicity,
            'data_points_analyzed': len(severities)
        }
    
    def _calculate_trend(self, severities: np.ndarray) -> str:
        """Calculate overall trend"""
        if len(severities) < 3:
            return 'insufficient_data'
        
        # Simple linear regression slope
        x = np.arange(len(severities))
        slope = np.polyfit(x, severities, 1)[0]
        
        if slope < -0.01:
            return 'improving'
        elif slope > 0.01:
            return 'worsening'
        else:
            return 'stable'
    
    def _detect_flares(self, severities: np.ndarray) -> Dict:
        """Detect flare events in the progression"""
        if len(severities) < 5:
            return {'count': 0, 'events': []}
        
        mean = np.mean(severities)
        std = np.std(severities)
        threshold = mean + 1.5 * std
        
        flares = []
        in_flare = False
        flare_start = 0
        
        for i, sev in enumerate(severities):
            if sev > threshold and not in_flare:
                in_flare = True
                flare_start = i
            elif sev <= threshold and in_flare:
                in_flare = False
                flares.append({
                    'start_index': flare_start,
                    'end_index': i - 1,
                    'duration': i - flare_start,
                    'peak_severity': float(np.max(severities[flare_start:i]))
                })
        
        return {
            'count': len(flares),
            'threshold': float(threshold),
            'events': flares
        }
    
    def _detect_periodicity(self, severities: np.ndarray) -> Dict:
        """Detect periodic patterns using autocorrelation"""
        if len(severities) < 14:
            return {'detected': False, 'reason': 'insufficient_data'}
        
        # Calculate autocorrelation
        n = len(severities)
        mean = np.mean(severities)
        var = np.var(severities)
        
        if var == 0:
            return {'detected': False, 'reason': 'no_variance'}
        
        autocorr = []
        for lag in range(1, min(n // 2, 60)):
            corr = np.mean((severities[:-lag] - mean) * (severities[lag:] - mean)) / var
            autocorr.append((lag, corr))
        
        # Find peaks in autocorrelation
        peaks = [(lag, corr) for lag, corr in autocorr if corr > 0.3]
        
        if peaks:
            best_period = peaks[0][0]
            return {
                'detected': True,
                'period_days': best_period,
                'correlation_strength': float(peaks[0][1]),
                'pattern_type': self._classify_period(best_period)
            }
        
        return {'detected': False, 'reason': 'no_significant_periodicity'}
    
    def _classify_period(self, period: int) -> str:
        """Classify the type of periodic pattern"""
        if period <= 7:
            return 'weekly'
        elif period <= 14:
            return 'biweekly'
        elif period <= 35:
            return 'monthly'
        else:
            return 'seasonal'

print("ProgressionPredictor loaded successfully")

## 4. Generate Synthetic Data and Demo

Demonstrate the ESN progression predictor with synthetic patient data.

In [ ]:
def generate_synthetic_progression(
    condition: str,
    days: int = 90,
    initial_severity: float = 0.7,
    treatment_start_day: int = 14,
    add_noise: bool = True
) -> PatientProgression:
    """Generate synthetic progression data for testing"""
    data_points = []
    base_date = datetime(2024, 1, 1)
    
    severity = initial_severity
    
    for day in range(days):
        current_date = base_date + timedelta(days=day)
        treatment_active = day >= treatment_start_day
        
        # Simulate progression
        if treatment_active:
            # Treatment effect
            severity *= 0.98  # 2% improvement per day
        else:
            # Natural fluctuation
            severity += np.random.normal(0, 0.02)
        
        # Add periodic flare (every ~28 days)
        if day % 28 < 3 and np.random.random() < 0.5:
            severity += 0.15
        
        # Add random noise
        if add_noise:
            severity += np.random.normal(0, 0.03)
        
        # Clamp severity
        severity = np.clip(severity, 0.05, 0.95)
        
        # Environmental factors
        month = current_date.month
        humidity = 0.5 + 0.2 * np.sin(2 * np.pi * month / 12)
        temperature = 0.5 + 0.3 * np.sin(2 * np.pi * (month - 3) / 12)
        
        data_points.append(ProgressionDataPoint(
            timestamp=current_date,
            severity=float(severity),
            treatment_active=treatment_active,
            treatment_type='topical_retinoid' if treatment_active else None,
            environmental_factors={
                'humidity': humidity,
                'temperature': temperature,
                'stress': np.random.uniform(0.2, 0.5)
            }
        ))
    
    return PatientProgression(
        patient_id='synthetic-patient-001',
        condition=condition,
        data_points=data_points
    )

# Generate synthetic data
progression = generate_synthetic_progression(
    condition='acne_vulgaris',
    days=60,
    initial_severity=0.75
)

print(f"Generated {len(progression.data_points)} data points")
print(f"Condition: {progression.condition}")
print(f"Date range: {progression.data_points[0].timestamp.date()} to {progression.data_points[-1].timestamp.date()}")

In [ ]:
# Train the predictor
predictor = ProgressionPredictor({
    'reservoir_size': 300,
    'spectral_radius': 0.9,
    'leaking_rate': 0.3
})

predictor.train(progression)

# Predict future progression
print("\n" + "=" * 60)
print("PREDICTING NEXT 30 DAYS")
print("=" * 60)

predictions = predictor.predict_progression(
    progression,
    horizon_days=30,
    treatment_plan={'active': True}
)

print("\nPredictions (every 5 days):")
for i in range(0, 30, 5):
    pred = predictions[i]
    print(f"  Day {i+1}: Severity={pred.predicted_severity:.3f} "
          f"(Confidence: {pred.confidence:.2f}, CI: [{pred.lower_bound:.3f}, {pred.upper_bound:.3f}])")

In [ ]:
# Analyze patterns
print("\n" + "=" * 60)
print("PATTERN ANALYSIS")
print("=" * 60)

patterns = predictor.analyze_patterns(progression)

print(f"\nStatistics:")
print(f"  Mean severity: {patterns['statistics']['mean_severity']:.3f}")
print(f"  Std deviation: {patterns['statistics']['std_severity']:.3f}")
print(f"  Trend: {patterns['statistics']['trend']}")

print(f"\nFlare Detection:")
print(f"  Flares detected: {patterns['flares']['count']}")

print(f"\nPeriodicity:")
if patterns['periodicity']['detected']:
    print(f"  Period: {patterns['periodicity']['period_days']} days")
    print(f"  Pattern type: {patterns['periodicity']['pattern_type']}")
else:
    print(f"  No significant periodicity detected")

In [ ]:
# Estimate healing time
print("\n" + "=" * 60)
print("HEALING TIME ESTIMATION")
print("=" * 60)

healing_estimate = predictor.estimate_healing_time(
    progression,
    target_severity=0.2,
    max_days=90
)

print(f"\nTarget severity: {healing_estimate['target_severity']}")
if healing_estimate['estimated_days']:
    print(f"Estimated days to target: {healing_estimate['estimated_days']}")
    print(f"Target date: {healing_estimate['target_date']}")
    print(f"Confidence: {healing_estimate['confidence']:.2f}")
else:
    print(f"Status: {healing_estimate['status']}")
    print(f"Final predicted severity: {healing_estimate.get('final_predicted_severity', 'N/A')}")

## 5. Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def plot_progression_with_predictions(
    progression: PatientProgression,
    predictions: List[ProgressionPrediction]
):
    """Plot historical progression with future predictions"""
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Historical data
    hist_dates = [dp.timestamp for dp in progression.data_points]
    hist_severities = [dp.severity for dp in progression.data_points]
    ax.plot(hist_dates, hist_severities, 'b-', linewidth=2, label='Historical', marker='o', markersize=3)
    
    # Treatment start indicator
    for i, dp in enumerate(progression.data_points):
        if dp.treatment_active and (i == 0 or not progression.data_points[i-1].treatment_active):
            ax.axvline(x=dp.timestamp, color='green', linestyle='--', alpha=0.7, label='Treatment Start')
            break
    
    # Predictions
    pred_dates = [p.timestamp for p in predictions]
    pred_severities = [p.predicted_severity for p in predictions]
    pred_lower = [p.lower_bound for p in predictions]
    pred_upper = [p.upper_bound for p in predictions]
    
    ax.plot(pred_dates, pred_severities, 'r-', linewidth=2, label='Predicted', marker='s', markersize=3)
    ax.fill_between(pred_dates, pred_lower, pred_upper, color='red', alpha=0.2, label='95% CI')
    
    # Formatting
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Severity (0-1)', fontsize=12)
    ax.set_title(f'Skin Condition Progression: {progression.condition.replace("_", " ").title()}', fontsize=14)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1)
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

# Plot the progression
plot_progression_with_predictions(progression, predictions)

## 6. Gateway Integration

In [ ]:
import requests

class ESNGatewayClient:
    """Client for ESN progression prediction through RegimAI Gateway"""
    
    def __init__(self, gateway_url: str, api_key: str):
        self.gateway_url = gateway_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            "X-API-Key": api_key,
            "Content-Type": "application/json"
        }
    
    def predict_progression(
        self,
        condition: str,
        patient_id: str,
        historical_data: List[Dict],
        prediction_horizon_days: int = 30
    ) -> Dict:
        """Request progression prediction from gateway"""
        payload = {
            "condition": condition,
            "patient_id": patient_id,
            "historical_data": historical_data,
            "prediction_horizon_days": prediction_horizon_days,
            "include_confidence_intervals": True
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/esn/predict",
            headers=self.headers,
            json=payload
        )
        
        return response.json()
    
    def analyze_patterns(
        self,
        condition: str,
        patient_id: str,
        historical_data: List[Dict]
    ) -> Dict:
        """Request pattern analysis from gateway"""
        payload = {
            "condition": condition,
            "patient_id": patient_id,
            "historical_data": historical_data
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/esn/patterns/analyze",
            headers=self.headers,
            json=payload
        )
        
        return response.json()
    
    def estimate_healing_time(
        self,
        condition: str,
        patient_id: str,
        historical_data: List[Dict],
        target_severity: float = 0.1
    ) -> Dict:
        """Request healing time estimation from gateway"""
        payload = {
            "condition": condition,
            "patient_id": patient_id,
            "historical_data": historical_data,
            "target_severity": target_severity
        }
        
        response = requests.post(
            f"{self.gateway_url}/cognitive/esn/healing/estimate",
            headers=self.headers,
            json=payload
        )
        
        return response.json()

print("ESN Gateway Client ready")
print(f"Gateway URL: {GATEWAY_URL}")

## Summary

This lab demonstrates:

1. **Echo State Networks**: Reservoir computing for time series prediction
2. **Progression Modeling**: Predicting skin condition severity over time
3. **Pattern Analysis**: Detecting flares, trends, and periodicity
4. **Healing Estimation**: Predicting time to recovery
5. **Gateway Integration**: API access through RegimAI Gateway

### Integration with SkinTwin Architecture

- **AtomSpace**: Store progression patterns as temporal relationships
- **PLN**: Reason about treatment efficacy from progression data
- **MOSES**: Optimize treatments based on predicted responses
- **ECAN**: Prioritize attention on high-risk progression patterns